## Preconfigurations

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import folium

## Data cleansing

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
table_file = soup.find("table", class_="wikitable sortable")

rows = table_file.findAll("tr")
with open('data/postcode_canada_wiki.csv', 'wt+') as f:
    writer = csv.writer(f)
    for row in rows:
        csv_row=[]
        for cell in row.findAll(['td', 'th']):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)   

In [3]:
temp_df = pd.read_csv('data/postcode_canada_wiki.csv')
# rename the column names
temp_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
# clean the column values
temp_df['Neighborhood'].replace("\\n", '', regex = True, inplace = True)
# There are column values with "Not assignned", change it to np.nan for good
temp_df["Borough"].replace("Not assigned", np.nan, inplace = True)
# Shape 
print("Shape: ", temp_df.shape[0])
temp_df.head(20)

Shape:  288


,PostalCode,Borough,Neighborhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,NaN,Not assigned


In [4]:
# delete rows that have np.nan values in 'Borough'
temp_df = temp_df[temp_df['Borough'].notna()]
#temp_df = temp_df[temp_df['Neighborhood'].notna()]
# reset index
temp_df = temp_df.reset_index(drop = True)
# Shape 
print("Shape: ", temp_df.shape[0])
temp_df.head()

Shape:  211


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
# save another dataframe which have the column 'Neighborhood' and can be joined back later
temp_df_backup = temp_df.copy()
temp_df_backup = temp_df_backup.drop(columns = "Neighborhood").set_index("PostalCode")
print("Shape: ",temp_df_backup.shape)
temp_df_backup.head()

Shape:  (211, 1)


,Borough
PostalCode,
M3A,North York
M4A,North York
M5A,Downtown Toronto
M5A,Downtown Toronto
M6A,North York


In [6]:
# For temp_df, group by PostalCode, and merge the rows with "Neighborhood", separated with ','
temp_df = pd.DataFrame(temp_df.groupby(['PostalCode'])['Neighborhood'].agg(", ".join))
temp_df = temp_df.reset_index()
print("Shape: ", temp_df.shape)
temp_df.head(20)

Shape:  (103, 2)


,PostalCode,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,"Clairlea, Golden Mile, Oakridge"
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,"Birch Cliff, Cliffside West"


In [7]:
# merge the 2 dataframes
temp_merged = temp_df.join(temp_df_backup, on='PostalCode').drop_duplicates()
# temp_merged[temp_merged['PostalCode'] == 'M5A'] # OK, checked
temp_merged.head(20)

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,"Birch Cliff, Cliffside West",Scarborough


In [8]:
# replace the "not assigned" Neighborhood name with it's corrsponding 'Borough' name
temp_index = temp_merged.loc[temp_merged['Neighborhood'] == 'Not assigned'].index
temp_merged.loc[temp_merged['Neighborhood'] == 'Not assigned']
temp_merged.loc[temp_index, 'Neighborhood'] = temp_merged.loc[temp_index, 'Borough']
temp_merged.loc[temp_index,:]

,PostalCode,Neighborhood,Borough
85,M7A,Queen's Park,Queen's Park


In [9]:
# Re-arrange the columns to make the order consistent
temp_list = [temp_merged.columns.values[0], temp_merged.columns.values[2], temp_merged.columns.values[1]]
temp_merged = temp_merged[temp_list]

In [10]:
# print out the Shape
print("The shape of DataFrame is {}.".format(temp_merged.shape))

The shape of DataFrame is (103, 3).


## Coordinates

Now our job is to get latitude and longitude for each row

### Coordinate for a single row

First we run a simple test to make sure, we know how to get coordinate for one of the rows

In [11]:
temp_merged.loc[0,'PostalCode']

'M1B'

In [12]:
43.6449033 -79.3818364

-35.736933099999995

In [13]:
# address = '102 North End Ave, New York, NY'
# address = '来福士广场, 上海市'
address = '{}, Toronto, Ontario'.format(temp_merged.loc[0,'PostalCode'])
geolocator = Nominatim(user_agent='yanyuan_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


Ok now it's a group thing

In [14]:
# define a function to get coordinates
def get_coord(postal_code):
    address = '{}, Toronto, Ontario'.format(temp_merged.loc[postal_code,'PostalCode'])
    location = Nominatim(user_agent='yanyuan_agent').geocode(address)
    lat = location.latitude
    long = location.longitude
    return lat, long

In [15]:
# create an empty DataFrame to store the coordinates
temp_num = len(temp_merged)
#coords = np.zeros()
lats = np.zeros(temp_num)
longs = np.zeros(temp_num)
for n in np.arange(0, temp_num):
    temp = get_coord(n)
    lats[n] = temp[0]
    longs[n] = temp[1]  

AttributeError: 'NoneType' object has no attribute 'latitude'

The problem is

1. `geocoder.google()` always returns `ACCESS DENIED`, and  `geocode()` does not return all coordinates in our dataframe

1. I'm sure the previous code works, but in order to finish the assignment, we have to download directly the coordinates from coursera website

In [16]:
# ! wget -O data/Geospatial_data.csv http://cocl.us/Geospatial_data

In [17]:
temp_coord = pd.read_csv('data/Geospatial_data.csv')
temp_coord.columns = ['PostalCode', 'Latitude', 'Longitude']
temp_coord = temp_coord.set_index('PostalCode')



temp_coord.head()
temp_merged = temp_df.join(temp_df_backup, on='PostalCode').drop_duplicates()

In [18]:
temp_merged.join(temp_coord, on='PostalCode')

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
5,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476
9,M1N,"Birch Cliff, Cliffside West",Scarborough,43.692657,-79.264848
